In [1]:
import inspect
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109
%load_ext autoreload
%autoreload 2
from AP_class import alch_deriv,first_deriv_nuc_nuc,second_deriv_nuc_nuc
from AP_symmetry import benz_Symm

In [18]:
benz_atom=""" 
C         6.95191249e-13  2.61173060e+00 -1.67580194e-08 ;
H         2.83052087e-13  4.63941350e+00 -1.15976219e-08 ;
C         2.26182505e+00  1.30586530e+00 -2.37508246e-08 ; 
H         4.01784995e+00  2.31970675e+00 -9.84687205e-09 ; 
C         2.26182505e+00 -1.30586530e+00  8.26065094e-09 ; 
H         4.01784995e+00 -2.31970675e+00  5.83890508e-08 ; 
C        -8.45954456e-13 -2.61173060e+00 -1.55922267e-08 ; 
H         1.49445304e-12 -4.63941350e+00 -3.26774154e-08 ; 
C        -2.26182505e+00 -1.30586530e+00 -8.28266973e-09 ; 
H        -4.01784995e+00 -2.31970675e+00  1.87236306e-09 ; 
C        -2.26182505e+00  1.30586530e+00  1.68546675e-08 ; 
H        -4.01784995e+00  2.31970675e+00  3.31289175e-08 ; 
"""
Benz = gto.M(atom=benz_atom,unit='bohr',basis={"H":"sto-3g","C":"cc-pvtz"},symmetry = 'D6v') #basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})

In [19]:
BenzHF=scf.RHF(Benz)
BenzHF.scf()

converged SCF energy = -230.743864823778


-230.74386482377844

In [4]:
from AP_symmetry import rotate_grad,rotate_matrix

In [5]:
from scipy.spatial.transform import Rotation as R

In [6]:
import itertools
%autoreload 2
from AP_class import DeltaV,parse_charge,alchemy_cphf_deriv,make_dP
from AP_class import APDFT_perturbator as AP
from pyscf.symm.basis import _ao_rotation_matrices as aorm

In [35]:
bs=benz_Symm(BenzHF)

In [34]:
ap=AP(BenzHF,sites=[0,2,4],symm=bs)

In [36]:
ap.dV(2)

array([[-3.82887830e-01,  1.04108126e-01, -1.95614373e-01, ...,
         3.81849016e-10, -7.08003163e-04, -1.91077231e-03],
       [ 1.04108126e-01, -3.82217390e-01, -2.12023178e-01, ...,
         6.24123064e-10, -1.27862450e-02, -1.82845297e-02],
       [-1.95614373e-01, -2.12023178e-01, -3.82887628e-01, ...,
         1.13243010e-09, -8.95651133e-03, -9.00532695e-03],
       ...,
       [ 3.81849016e-10,  6.24123064e-10,  1.13243010e-09, ...,
        -2.21225764e-01,  1.36152004e-10, -2.16730513e-10],
       [-7.08003163e-04, -1.27862450e-02, -8.95651133e-03, ...,
         1.36152004e-10, -2.26778349e-01, -1.62982574e-03],
       [-1.91077231e-03, -1.82845297e-02, -9.00532695e-03, ...,
        -2.16730513e-10, -1.62982574e-03, -1.57208230e-01]])

In [11]:
dV=DeltaV(Benz,[[0],[1]])

In [13]:
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [24]:
ang=np.pi/3
rm_p=np.asarray([[np.cos(ang),-np.sin(ang),0],\
                 [np.sin(ang),np.cos(ang),0],\
                [0,0,1]])
s_rm,p_rm,d_rm,f_rm=aorm(Benz,rm_p)

In [23]:
Benz.ao_labels()

['0 C 1s    ',
 '0 C 2s    ',
 '0 C 3s    ',
 '0 C 4s    ',
 '0 C 2px   ',
 '0 C 2py   ',
 '0 C 2pz   ',
 '0 C 3px   ',
 '0 C 3py   ',
 '0 C 3pz   ',
 '0 C 4px   ',
 '0 C 4py   ',
 '0 C 4pz   ',
 '0 C 3dxy  ',
 '0 C 3dyz  ',
 '0 C 3dz^2 ',
 '0 C 3dxz  ',
 '0 C 3dx2-y2',
 '0 C 4dxy  ',
 '0 C 4dyz  ',
 '0 C 4dz^2 ',
 '0 C 4dxz  ',
 '0 C 4dx2-y2',
 '0 C 4fy^3 ',
 '0 C 4fxyz ',
 '0 C 4fyz^2',
 '0 C 4fz^3 ',
 '0 C 4fxz^2',
 '0 C 4fzx^2',
 '0 C 4fx^3 ',
 '1 H 1s    ',
 '2 C 1s    ',
 '2 C 2s    ',
 '2 C 3s    ',
 '2 C 4s    ',
 '2 C 2px   ',
 '2 C 2py   ',
 '2 C 2pz   ',
 '2 C 3px   ',
 '2 C 3py   ',
 '2 C 3pz   ',
 '2 C 4px   ',
 '2 C 4py   ',
 '2 C 4pz   ',
 '2 C 3dxy  ',
 '2 C 3dyz  ',
 '2 C 3dz^2 ',
 '2 C 3dxz  ',
 '2 C 3dx2-y2',
 '2 C 4dxy  ',
 '2 C 4dyz  ',
 '2 C 4dz^2 ',
 '2 C 4dxz  ',
 '2 C 4dx2-y2',
 '2 C 4fy^3 ',
 '2 C 4fxyz ',
 '2 C 4fyz^2',
 '2 C 4fz^3 ',
 '2 C 4fxz^2',
 '2 C 4fzx^2',
 '2 C 4fx^3 ',
 '3 H 1s    ',
 '4 C 1s    ',
 '4 C 2s    ',
 '4 C 3s    ',
 '4 C 4s    ',
 '4 C 

In [29]:
p_idxs=[i for i,elem in enumerate(Benz.ao_labels()) if  "px" in elem]
d_idxs=[i for i,elem in enumerate(Benz.ao_labels()) if  "dxy" in elem]
f_idxs=[i for i,elem in enumerate(Benz.ao_labels()) if  "fy^3" in elem]

In [ ]:
ang=np.pi/3
rm_p=np.asarray([[np.cos(ang),-np.sin(ang),0],\
                 [np.sin(ang),np.cos(ang),0],\
                [0,0,1]])
s_rm,p_rm,d_rm,f_rm=aorm(Benz,rm_p)

In [31]:
rm=np.eye(Benz.nao)
for i in p_idxs:
    rm[i:i+3,i:i+3]=p_rm
for i in d_idxs:
    rm[i:i+5,i:i+5]=d_rm
for i in f_idxs:
    rm[i:i+7,i:i+7]=f_rm

In [37]:
np.allclose(ap.dV(0),rotate_matrix(rm.T@ap.dV(2)@rm,Benz,0,ref_site=2),atol=1e-3)

True

In [38]:
dm0=BenzHF.make_rdm1()

In [41]:
np.allclose(dm0,rotate_matrix(rm.T@dm0@rm,Benz,0,ref_site=2),atol=1e-4)

True

In [131]:
pt=Benz.aoslice_by_atom()[0,-2]
rpt=Benz.aoslice_by_atom()[2,-2]

In [ ]:
BenzHF.make_rdm1()[pt:pt+5,pt:pt+5],BenzHF.make_rdm1()[rpt:rpt+5,rpt:rpt+5]

In [ ]:
np.cos(np.pi/3)*0.0191,np.sin(np.pi/3)*0.0191

In [ ]:
np.array((0,0,1))

In [ ]:
coords=Benz.atom_coords()

In [ ]:
#symm.Dmatrix.get_euler_angles(coords[0],coords[2])

In [ ]:
#symm.geom.detect_symm(Benz.atom_and_coords())

In [ ]:
AP.mo1(2).shape

In [ ]:
bsym=benz_Symm(Benz)

In [ ]:
bsym.eqs

In [ ]:
coords=Benz.atom_coords()

In [ ]:
coords[::]

In [ ]:
rotate_grad(bsym.eqs[4]['op'].apply(coords),4,ref_site=0)

In [ ]:
rotate_grad(bsym.eqs[6]['op'].apply(AP.af(0)),6)

In [ ]:
AP.af(6)

In [ ]:
np.allclose(rotate_grad(bsym.eqs[6]['op'].apply(AP.af(0)),6),AP.af(6))